## Práticas Atuarias
### Trabalho 1 e 2

In [5]:
# Importando bibliotecas 

import pandas as pd
import numpy as np
import os
import datetime
from dateutil.relativedelta import relativedelta

# Lendo e tratando a base de dados

In [42]:
# Classe que realiza as operações nos assistidos do plano

class Assistido:
    def __init__(self, num_registro, sexo, data_admissão, data_nascimento_titular, idade_titular, salário, data_nascimento_cônjuge, idade_cônjuge, data_nascimento_filho_a, idade_filho_a, data_nascimento_filho_b, idade_filho_b, data_nascimento_filho_c, idade_filho_c, **kwargs):
        self.num_registro = num_registro
        self.sexo = sexo
        self.data_admissão = data_admissão
        self.data_nascimento_titular = data_nascimento_titular
        self.idade_titular = idade_titular
        self.salario = salário
        self.data_nascimento_conjuge = data_nascimento_cônjuge
        self.idade_conjuge = idade_cônjuge
        self.data_nascimento_filho_a = data_nascimento_filho_a
        self.idade_filho_a = idade_filho_a
        self.data_nascimento_filho_b = data_nascimento_filho_b
        self.idade_filho_b = idade_filho_b
        self.data_nascimento_filho_c = data_nascimento_filho_c
        self.idade_filho_c = idade_filho_c
        
    def __str__(self):
        return str(self.num_registro)
        
    def check_filho_a(self, idade):
        if pd.isnull(idade):
            return True
        else:
            if idade <= (self.idade_titular - 12):
                if idade >= 0:
                    return True

            return False

    def check_filho_b(self, idade, idade_ref):
        if pd.isnull(idade):
            return True
        else:
            if idade >= 0:
                if idade <= (idade_ref -1):
                    return True

            return False
        
    def check_filho_c(self, idade, idade_ref):
        if pd.isnull(idade):
            return True
        else:
            if idade >= 0:
                if idade <= (idade_ref):
                    return True

            return False

In [26]:
def clean_cols(col_name):
    """Trata as colunas para ficarem padronizadas"""
    # trata casos especiais
    if col_name == 'Nº Registro':
        return 'num_registro'
    elif 'Data Nascito.' in col_name:
        suffix = col_name.split('Data Nascito.')[-1].strip()
        return 'data_nascimento_' + suffix.lower().replace(' ', '_')

    # tratamento geral
    else:
        return col_name.lower().replace(' ', '_')
    
def get_age(x, today=datetime.datetime.today):
    """Retorna a idade relativa a data passada"""
    try:
        diff = relativedelta(today(), x)
        return diff.years
    except AssertionError:
        return np.nan


# Le o arquivo original
df_original = pd.read_excel("201 Cond Trab 1 e 2 Eduardo W abr20.xls", sheet_name="Dados", header=2)
    
# Faz uma cópia e aplica a função clean_cols
df_cleanse_columns = df_original.copy()
df_cleanse_columns.columns = [clean_cols(col) for col in df_original.columns]

# faz uma cópia do DataFrame com as colunas padronizadas e transforma colunas de data em datetime objects
df = df_cleanse_columns.copy()
for i, col in enumerate(df_cleanse_columns.columns):
    if 'unnamed' in col:
        df[df_cleanse_columns.iloc[:, i-1].name] = pd.to_datetime(
            df_cleanse_columns[col].astype(str) + df_cleanse_columns.iloc[:, i-1].replace(to_replace=0, value=1).astype(str), # SUBSTITUI MES ZERO POR 1 
            format='%Y%m', 
            errors='coerce'
        )
        df = df.drop(col, axis='columns')
        
# limpa linhas vazias
df = df.dropna(axis='index', how='all')

# adiciona a coluna de idades ao DataFrame
for i, col in enumerate(df.columns):
    if 'data_nascimento' in col:
        idades = df[col].apply(get_age)
        col_name = col.replace('data_nascimento', 'idade')
        loc = df.columns.get_loc(col)
        df.insert(loc=loc+1, column=col_name, value=idades)

df.head()

,num_registro,sexo,data_admissão,data_nascimento_titular,idade_titular,salário,data_nascimento_cônjuge,idade_cônjuge,data_nascimento_filho_a,idade_filho_a,data_nascimento_filho_b,idade_filho_b,data_nascimento_filho_c,idade_filho_c
1,1201.0,F,2016-12-01,1986-01-01,34,3529.5448,1986-01-01,34.0,2012-08-01,7.0,2019-06-01,0.0,2021-01-01,0.0
2,1202.0,M,2008-06-01,1978-04-01,42,3280.3036,1975-01-01,45.0,2014-05-01,5.0,1988-03-01,32.0,NaT,NaN
3,1203.0,M,2010-07-01,1975-09-01,44,1786.8844,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
4,1204.0,M,2014-06-01,1972-11-01,47,1505.2966,1969-01-01,51.0,2014-09-01,5.0,2016-07-01,3.0,2005-02-01,15.0
5,1205.0,M,2005-10-01,1970-06-01,49,4601.9512,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN


## Tratamentos nos dados

### Problemas filho "A"
1. Idade menor que 0
2. Idade maior que do titular menos a idade reprodutiva mínima

#### A correção se dá por substituir a idade que não se encaixou nos critérios pela idade do titular menos a idade média de obtenção do primeiro filho, exceto por um caso em que o resultado dessa operação resultou em idade negativa do filho, esse caso foi corrigindo da idade do titular a idade mínima reprodutiva

In [27]:
# DF apenas contendo registros que obedecem todos os critérios acima
df_filhoA = df[(df['idade_filho_a'] < df['idade_titular']-12) & (df['idade_filho_a'] > 0)]
diferenca_titular = df_filhoA['idade_titular'] - df_filhoA['idade_filho_a'] # Idade media que os titulares têm o primeiro filho
diferenca_media_titular = diferenca_titular.mean()

print(f'Dos {diferenca_titular.shape[0]} que possuem filhos, idade média quando o primeiro filho nasceu é: {round(diferenca_media_titular, ndigits=0)} anos\n')

def corrige_filho_a(row, diff):
    registro = Assistido(**row.to_dict())
    if registro.check_filho_a(registro.idade_filho_a) == False:
        #print(registro.idade_titular, registro.idade_filho_a, registro.idade_titular - diff)
        nova_idade = round(registro.idade_titular - diff, ndigits=0)
        if nova_idade <= 0:
            print(f'O colaborador com número de registro {registro.num_registro} ficou com a idade do filho negativo, nesse caso foi aplicado sobre a idade do titular a idade mínima de reprodução para corrigir a idade do filho A\n')
            return registro.idade_titular - 12
        else:
            return nova_idade
    else:
        return registro.idade_filho_a
    
def check_correcao_a(row, new_col):
    registro = Assistido(**row.to_dict())
    if registro.check_filho_a(row[new_col]) == False:
        return 'Erro'
    else:
        return 'Ok'
    

df['idade_corrigida_a'] = df.apply(corrige_filho_a, axis='columns', diff=diferenca_media_titular)
valido_apos_correcao = df.apply(check_correcao_a, axis='columns', new_col='idade_corrigida_a')
print('Após a correção todos os registros passaram no teste:\n',valido_apos_correcao.value_counts())

Dos 846 que possuem filhos, idade média quando o primeiro filho nasceu é: 25.0 anos

O colaborador com número de registro 1812.0 ficou com a idade do filho negativo, nesse caso foi aplicado sobre a idade do titular a idade mínima de reprodução para corrigir a idade do filho A

Após a correção todos os registros passaram no teste:
 Ok    1300
dtype: int64


### Problemas filho "B"
1. Idade menor que 0
2. Idade menor que a do filho "A" menos 1 ano

\#### A correção se dá por substituir a idade que não se encaixou nos critérios pela idade do titular menos a idade média de obtenção do primeiro filho, exceto por um caso em que o resultado dessa operação resultou em idade negativa do filho, esse caso foi corrigindo da idade do titular a idade mínima reprodutiva

In [32]:
# DF apenas contendo registros que obedecem todos os critérios acima
df_filhoB = df[(df['idade_filho_b'] >= 0) & (df['idade_filho_b'] <= (df['idade_corrigida_a']-1))]
diferenca_filhosB = df_filhoB['idade_corrigida_a'] - df_filhoB['idade_filho_b'] # diferença de idade entre filhos "A" e "B"
diferenca_media_filhosB = diferenca_filhosB.mean()

print(f'Dos {df_filhoB.shape[0]} que possuem pelo menos 2 filhos, a diferença média de idade entre os filhos "A" e "B" é de: {round(diferenca_media_filhosB, ndigits=0)} anos\n')

def corrige_filho_b(row, idade_ref, diff):
    registro = Assistido(**row.to_dict())
    if registro.check_filho_b(registro.idade_filho_b, row[idade_ref]) == False:
        #print(registro.idade_titular, registro.idade_filho_a, registro.idade_titular - diff)
        nova_idade = round(row[idade_ref] - diff, ndigits=0)
        if nova_idade <= 0:
            print(f'O colaborador com número de registro {registro.num_registro} ficou com a idade do filho "B" negativo, nesse caso foi usado o intervalo entre a idade do filho A e zero')
            return round(row[idade_ref]/2, ndigits=0)
        else:
            return nova_idade
    else:
        return registro.idade_filho_b
    
def check_correcao_b(row, new_col, idade_ref):
    registro = Assistido(**row.to_dict())
    if registro.check_filho_b(row[new_col], row[idade_ref]) == False:
        return 'Erro'
    else:
        return 'Ok'
    

df['idade_corrigida_b'] = df.apply(corrige_filho_b, axis='columns', idade_ref='idade_corrigida_a',diff=diferenca_media_filhosB)
valido_apos_correcao = df.apply(check_correcao_b, axis='columns', new_col='idade_corrigida_b', idade_ref='idade_corrigida_a')
print('Após a correção todos os registros passaram no teste:\n',valido_apos_correcao.value_counts())

Dos 601 que possuem pelo menos 2 filhos, a diferença média de idade entre os filhos "A" e "B" é de: 6.0 anos

O colaborador com número de registro 1202.0 ficou com a idade do filho "B" negativo, nesse caso foi usado o intervalo entre a idade do filho A e zero
O colaborador com número de registro 1236.0 ficou com a idade do filho "B" negativo, nesse caso foi usado o intervalo entre a idade do filho A e zero
O colaborador com número de registro 1242.0 ficou com a idade do filho "B" negativo, nesse caso foi usado o intervalo entre a idade do filho A e zero
O colaborador com número de registro 1243.0 ficou com a idade do filho "B" negativo, nesse caso foi usado o intervalo entre a idade do filho A e zero
O colaborador com número de registro 1244.0 ficou com a idade do filho "B" negativo, nesse caso foi usado o intervalo entre a idade do filho A e zero
O colaborador com número de registro 1268.0 ficou com a idade do filho "B" negativo, nesse caso foi usado o intervalo entre a idade do filho

### Problemas filho "C"
1. Idade menor que 0
2. Idade menor que a do filho "B" menos 1 ano

\#### A correção se dá por substituir a idade que não se encaixou nos critérios pela idade do titular menos a idade média de obtenção do primeiro filho, exceto por um caso em que o resultado dessa operação resultou em idade negativa do filho, esse caso foi corrigindo da idade do titular a idade mínima reprodutiva

In [45]:
# DF apenas contendo registros que obedecem todos os critérios acima
df_filhoC = df[(df['idade_filho_c'] >= 0) & (df['idade_filho_c'] <= (df['idade_corrigida_b']-1))]
diferenca_filhosC = df_filhoC['idade_corrigida_b'] - df_filhoC['idade_filho_c'] # diferença de idade entre filhos "A" e "B"
diferenca_media_filhosC = diferenca_filhosC.mean()

print(f'Dos {df_filhoC.shape[0]} que possuem pelo menos 2 filhos, a diferença média de idade entre os filhos "B" e "C" é de: {round(diferenca_media_filhosC, ndigits=0)} anos\n')

def corrige_filho_c(row, idade_ref, diff):
    registro = Assistido(**row.to_dict())
    if registro.check_filho_c(registro.idade_filho_c, row[idade_ref]) == False:
        #print(registro.idade_titular, registro.idade_filho_a, registro.idade_titular - diff)
        nova_idade = round(row[idade_ref] - diff, ndigits=0)
        if nova_idade <= 0:
            print(f'O colaborador com número de registro {registro.num_registro} ficou com a idade do filho "C" negativo, nesse caso foi usado o intervalo entre a idade do filho B e zero')
            return round(row[idade_ref]/2, ndigits=0)
        else:
            return nova_idade
    else:
        return registro.idade_filho_c
    
def check_correcao_c(row, new_col, idade_ref):
    registro = Assistido(**row.to_dict())
    if registro.check_filho_c(row[new_col], row[idade_ref]) == False:
        return 'Erro'
    else:
        return 'Ok'
    

df['idade_corrigida_c'] = df.apply(corrige_filho_c, axis='columns', idade_ref='idade_corrigida_b',diff=diferenca_media_filhosC)
valido_apos_correcao = df.apply(check_correcao_c, axis='columns', new_col='idade_corrigida_c', idade_ref='idade_corrigida_b')
print('Após a correção todos os registros passaram no teste:\n',valido_apos_correcao.value_counts())

Dos 402 que possuem pelo menos 2 filhos, a diferença média de idade entre os filhos "B" e "C" é de: 5.0 anos

O colaborador com número de registro 1204.0 ficou com a idade do filho "C" negativo, nesse caso foi usado o intervalo entre a idade do filho B e zero
O colaborador com número de registro 1227.0 ficou com a idade do filho "C" negativo, nesse caso foi usado o intervalo entre a idade do filho B e zero
O colaborador com número de registro 1236.0 ficou com a idade do filho "C" negativo, nesse caso foi usado o intervalo entre a idade do filho B e zero
O colaborador com número de registro 1242.0 ficou com a idade do filho "C" negativo, nesse caso foi usado o intervalo entre a idade do filho B e zero
O colaborador com número de registro 1243.0 ficou com a idade do filho "C" negativo, nesse caso foi usado o intervalo entre a idade do filho B e zero
O colaborador com número de registro 1244.0 ficou com a idade do filho "C" negativo, nesse caso foi usado o intervalo entre a idade do filho